In [1]:
import opendatasets as od
import numpy as np
import pandas as pd

In [17]:
from keras.models import Model
from keras.backend import shape,int_shape,random_normal,exp
from keras.layers import Input,Dense,Flatten,Conv2D,Conv2DTranspose,Lambda,Reshape,BatchNormalization,LeakyReLU,Dropout
from keras.preprocessing.image import ImageDataGenerator

In [3]:
od.download('https://www.kaggle.com/datasets/jessicali9530/celeba-dataset')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading celeba-dataset.zip to .\celeba-dataset


100%|██████████| 1.33G/1.33G [28:10<00:00, 845kB/s]  


In [3]:
data_path = 'G:\Generative-deep-learning\Projects\celeb-face-vae\celeba-dataset\img_align_celeba'

In [4]:
input_dim = (128,128,3)
batch_size = 32

In [6]:
img_gen = ImageDataGenerator(rescale=1./255)
flow = img_gen.flow_from_directory(data_path, target_size=input_dim[:2], batch_size=batch_size,shuffle=True,
                                   class_mode='input',subset='training')

Found 202599 images belonging to 1 classes.


In [13]:
# network params
input_shape = input_dim
batch_size = 32
layers = [32,64,64,64]
latent_dim = 200


In [14]:
input_layer = Input(shape=input_shape,name='encoder_input_layer')

x = input_layer

for layer in layers:
    conv_layer = Conv2D(filters=layer, kernel_size=3,strides=2, padding='same')
    x = conv_layer(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.5)(x)
    x = LeakyReLU()(x)

shape_before_flatten = int_shape(x)

x = Flatten()(x)

mu = Dense(units=latent_dim, name = 'mu')(x)
log_var = Dense(units=latent_dim,name = 'log_var')(x)

encoder_mu_log_var =  Model(input_layer,(mu,log_var))

encoder_mu_log_var.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_layer (InputLaye  [(None, 128, 128, 3  0          []                               
 r)                             )]                                                                
                                                                                                  
 conv2d_9 (Conv2D)              (None, 64, 64, 32)   896         ['encoder_input_layer[0][0]']    
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 64, 64, 32)  128         ['conv2d_9[0][0]']               
 rmalization)                                                                                     
                                                                                            

In [18]:
def sampling(args):
    mu,log_var = args
    epsilon = random_normal(shape=shape(mu), mean=0, stddev=1)
    return mu + exp(log_var/2)*epsilon

In [19]:
encoder_output = Lambda(sampling, name = 'encoder_output')([mu,log_var])

encoder = Model(input_layer,encoder_output)

encoder.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_layer (InputLaye  [(None, 128, 128, 3  0          []                               
 r)                             )]                                                                
                                                                                                  
 conv2d_9 (Conv2D)              (None, 64, 64, 32)   896         ['encoder_input_layer[0][0]']    
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 64, 64, 32)  128         ['conv2d_9[0][0]']               
 rmalization)                                                                                     
                                                                                            

In [ ]:
# Decoder